# WER (Word Error Rate)

In [1]:
import werpy
import pandas as pd


In [7]:
ref = pd.read_csv("/Users/manavgarg/Documents/Nicholas Duran Project/911-Call-Analysis/Sample Files/sample_human_file.csv")
hyp = pd.read_csv("/Users/manavgarg/Documents/Nicholas Duran Project/911-Call-Analysis/Sample Files/sample_ai_file.csv")

In [8]:
unique_calls = set(hyp['CallName'].unique()).intersection(set(ref['CallName'].unique()))
len(unique_calls)

2

In [9]:
call_name = []
wer_score = []

In [11]:
for call in unique_calls:

  cur_ref = ref[ref['CallName'] == call]
  cur_hyp = hyp[hyp['CallName'] == call]

  cur_ref.sort_values(by='startutt', inplace=True)
  cur_hyp.sort_values(by='startutt', inplace=True)

  call_name.append(call)

  cur_ref_transcript = werpy.normalize( ' '.join(cur_ref['content']))
  cur_hyp_transcript =  werpy.normalize(' '.join(cur_hyp['content']))

  wer_score.append(werpy.wer(cur_ref_transcript, cur_hyp_transcript))



/var/folders/t6/f2wllc7119l6jdp9v4vrl6gr0000gn/T/ipykernel_21608/1711983494.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_ref.sort_values(by='startutt', inplace=True)
/var/folders/t6/f2wllc7119l6jdp9v4vrl6gr0000gn/T/ipykernel_21608/1711983494.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_hyp.sort_values(by='startutt', inplace=True)


In [15]:
print(sum(wer_score)/len(wer_score))

0.2194167155999217


In [13]:
from pyannote.core import Segment, Annotation, Timeline
from pyannote.metrics.diarization import DiarizationErrorRate
from pyannote.metrics.detection import DetectionErrorRate
from pyannote.metrics.diarization import JaccardErrorRate

In [17]:
ref.head()

,Unnamed: 0,CallName,channel,startutt,stoputt,content
0,0,01-TUC-18040600,operator,1.880,2.860,"Tuscon police, [xxx]"
1,1,01-TUC-18040600,operator,3.280,5.060,This is [xxx]
2,2,01-TUC-18040600,caller,4.965,5.745,I need somebody here.
3,3,01-TUC-18040600,operator,6.225,7.785,"Uh sir hold on one second, okay?"
4,4,01-TUC-18040600,operator,7.875,19.305,He saw a patient in the road on 8th and Park w...


In [40]:
def create_annotation(cur_df):

    annotation = Annotation()

    for index, row in cur_df.iterrows():
        annotation[Segment(float(row["startutt"]), float(row["stoputt"]))] = row["channel"]
    return annotation

In [48]:
ders = []
jers = []
detection_error_rates = []
for call in unique_calls:
   print(f"Call Name: {call}")
   cur_ref = ref[ref['CallName'] == call]
   cur_hyp = hyp[hyp['CallName'] == call]
   
   # Convert segments to pyannote.core.Annotation format
   reference = create_annotation(cur_ref)
   hypothesis = create_annotation(cur_hyp)


   # Diarization Error Rate (DER)
   der = DiarizationErrorRate()
   der_value = der(reference, hypothesis)
   ders.append(der_value)
   print(f'Diarization Error Rate (DER): {der_value:.4f}')
   
   # Jaccard Error Rate (JER)
   jer = JaccardErrorRate()
   jer_value = jer(reference, hypothesis)
   jers.append(jer_value)
   print(f'Jaccard Error Rate (JER): {jer_value:.4f}')

   # # False Alarm Rate and Missed Speech Rate
   # # We need to extract the timelines (support) from the reference and hypothesis annotations

   # Detection Error Rate metric calculates false alarm, missed detection, and overall detection error rate.
   detection_error_rate = DetectionErrorRate()
   detection_result = detection_error_rate(reference, hypothesis)
   detection_error_rates.append(detection_result)
   print(f'Detection Error Rate (JER): {detection_result:.4f}')
   print()

Call Name: 01-TUC-18040600
Diarization Error Rate (DER): 0.1991
Jaccard Error Rate (JER): 0.1830
Detection Error Rate (JER): 0.1886

Call Name: 287-PHX-2018-00886871-01Track1
Diarization Error Rate (DER): 0.2980
Jaccard Error Rate (JER): 0.2910
Detection Error Rate (JER): 0.2191



/opt/miniconda3/envs/911/lib/python3.9/site-packages/pyannote/metrics/utils.py:200: UserWarning: 'uem' was approximated by the union of 'reference' and 'hypothesis' extents.
  warnings.warn(


In [50]:
print(f"{sum(ders)/len(ders)}")

print(f"{sum(jers)/len(jers)}")

print(f"{sum(detection_error_rates)/len(detection_error_rates)}")

0.2485552540143669
0.23699163485105001
0.20381343673215344


In [34]:
cur_hyp

,Unnamed: 0,CallName,startutt,stoputt,channel,content
72,3311,287-PHX-2018-00886871-01Track1,2.520,4.674,operator,"Phoenix 911, where is your emergency?"
73,3312,287-PHX-2018-00886871-01Track1,5.214,30.444,caller,"Yes, how are you doing? Yes, I stay at Serena ..."
74,3313,287-PHX-2018-00886871-01Track1,32.824,35.724,operator,You keep cutting out a little bit. What apartm...
75,3314,287-PHX-2018-00886871-01Track1,38.904,40.240,caller,Serena village.
76,3315,287-PHX-2018-00886871-01Track1,40.392,57.144,operator,"Okay, but what unit is he in front of? Where i..."
77,3316,287-PHX-2018-00886871-01Track1,57.604,58.744,caller,I'm back inside.
78,3317,287-PHX-2018-00886871-01Track1,59.684,62.944,operator,"Now, what the guy looked like, was he white, b..."
79,3318,287-PHX-2018-00886871-01Track1,65.644,66.348,caller,Huh?
80,3319,287-PHX-2018-00886871-01Track1,66.476,70.024,operator,"Hello, sir. What did the guy look like? White,..."
81,3320,287-PHX-2018-00886871-01Track1,79.404,80.944,caller,People trying to kill him.
